In [ ]:
# 예제 : https://teddylee777.github.io/langchain/rag-tutorial/
#
# LangChain 설치 및 업데이트
#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [ ]:
import os

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [2]:
# 단계1: 폴더내 모든 문서 로딩
#!pip install unstructured

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="./doc_test/*.txt", show_progress=True)
docs = loader.load()

print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[1].page_content[:200]}")
print(f"\n[metadata]\n{docs[1].metadata}\n")

100%|██████████| 9/9 [00:08<00:00,  1.01it/s]

문서의 수: 9

[페이지내용]
대학교 학자금대출 제도

목적 : 대학생 자녀를 가진 임직원의 학자금을 대출함으로써 임직원의 복지향상, 근로의욕의 제고 및 장기근속 유도

도입기준

대학생 자녀를 가진 정규직 임직원

지급신청 : 부문장(본부는 본부장) 전결로 그룹웨어를 통해 신청

대학교 학자금대출 신청서(그룹웨어)

재학증명서, 입학증명서 중 1종 첨부

지원절차

그룹웨어를 통해 ‘

[metadata]
{'source': 'doc_test/06.대학교 학자금 대출 지원 제도_20.02.01.txt'}



In [3]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

print(f'*chunk 수:{len(splits)}\n')
print(f'*splits[0]:\n{splits[0]}')


*chunk 수:40

*splits[0]:
page_content='경조사 지원 규정\n\n1. 적용대상\n\n계약직 사원을 포함한 임직원(고문 및 용역은 사장이 별도로 결정)\n\n2. 경조사 지원기준\n\n구 분\n\n내 역\n\n휴가(일)\n\n금액(원)\n\n기타\n\n경 사\n\n본인 결혼\n\n자녀 결혼\n\n형제자매결혼\n\n자녀 출산(배우자)\n\n부모 회갑\n\n배우자 부모회갑\n\n부모고희(칠순)\n\n배우자부모고희\n\n5\n\n1\n\n1\n\n10\n\n1\n\n1\n\n1\n\n1\n\n500,000\n\n300,000\n\n100,000\n\n100,000\n\n200,000\n\n200,000\n\n300,000\n\n300,000\n\n화환 지급\n\n화환 지급\n\n1회 분할 사용 가능\n\n조 사\n\n본인 사망\n\n배우자 사망\n\n부모 사망\n\n자녀 사망\n\n배우자 부모 사망\n\n형제자매 사망\n\n조부모/외조부모 사망\n\n5\n\n5\n\n5\n\n5\n\n3\n\n3\n\n1,000,000\n\n1,000,000\n\n1,000,000\n\n1,000,000 500,000\n\n300,000\n\n200,000\n\n조화 지급\n\n조화 지급\n\n조화 지급\n\n조화 지급\n\n조화 지급\n\n조화 지급' metadata={'source': 'doc_test/03.경조사지원규정_12.01.01.txt'}


In [ ]:
# sqlalchemy 에러가 나면 업그레이드 해준다.
#!pip install --upgrade sqlalchemy

In [4]:
# 단계 3: 임베딩
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS, Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# 단계 3, 4: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
#vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

#vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

# 단계 5: 리트리버 생성(Create Retriever)
# 사용자의 질문(query) 에 부합하는 문서를 검색합니다.
# 유사도 높은 K 개의 문서를 검색합니다.
k = 3

# BM25 retriver 생성
# (Sparse) bm25 retriever and (Dense) faiss retriever 를 초기화 합니다.
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = k

# 임베딩 retiver 모델을 불러옴.
model_name = "bongsoo/kpf-sbert-v1"
#model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
custom_embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    #model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 임베딩 retriver 생성
#faiss_vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))
faiss_vectorstore = FAISS.from_documents(documents=splits, embedding=custom_embedding_model)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})

# bm25 + 임베딩 retriver 합쳐서 만듬.
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n\n".join(doc.page_content for doc in docs)


RAG-prompt:
input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [11]:
# 단계 8: 체인 생성(Create Chain)
# => 검색된 문서들을 출력함.
rag_chain = (
    {"context": bm25_retriever, "question": RunnablePassthrough()}
    | prompt
)

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "1박2일 서울 출장시 식비는 얼마인가요?"
response = rag_chain.invoke(question)

# 결과 출력
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

문서의 수: 9
[HUMAN]
1박2일 서울 출장시 식비는 얼마인가요?

[AI]
messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 1박2일 서울 출장시 식비는 얼마인가요? \nContext: [Document(page_content='12. 휴직자, 해직자에 대한 여비\\n\\n사무인계 또는 잔무정리등을 위하여 휴직 또는 해직직원 출장시 본직 또는 전직에 상당한 여비\\n\\n를 지급한다.\\n\\n13. 출장중 상병\\n\\n임직원이 출장중 부상 또는 질병으로 가족의 간호가 필요하다고 인정될 경우 가족 1인에 대하여\\n\\n왕복여비로 본인과 동일등급의 여비상당액을 지급하며, 의사의 진단서 등 이를 증명할 수 있는\\n\\n증빙서류에 의하여 연장체제중의 여비를 지급할 수 있다.\\n\\n14. 출장중 사망\\n\\n14.1 사망하였을 경우, 사망지부터 거주지까지의 유해 운구비를 지급한다.\\n\\n14.2 가족이 유해를 운구할 경우 가족 1인에 대하여 왕복여비로 본인과 동일등급의 여비상당액을\\n\\n지급한다.\\n\\n15. 국내출장여비\\n\\n12.1 지급기준\\n\\n별표 1에 따라, 일비와 식비는 정액을, 운임과 숙박비는 실비 정산한다.\\n\\n12.2 당일출장\\n\\n출장기간이 1일인 출장으로 출장지역에서 4시간 이상 업무수행시 출장여비를 지급한다.\\n\\n12.3 장기체제\\n\\n6.2. 규정에 따른다.', metadata={'source': 'doc_test/25.출장여비규정_16.04.01